#### CIE4604 Simulation and Visualization
# Module 2 Satellite Orbits - Exercise 1

**Hans van der Marel, 15 November 2020** (updated 15 November 2021)

In this exercise you download NORAD Two-Line Elements (TLE) to compute and visualize orbits for Earth observations satellites in Low and Medium Earth orbits.

## Explore the crsutil.py and tleplot.py Python modules

Download `CIE4604-M2-python.zip` from Brightspace and unzip this in your current working directory. This should give
you two Python files: `crsutil.py` and `tleplot.py`. the download also contains a file with coastlines and an example TLE file.

For this Jupyter notebook to work, the Jupyter notebook should be in the same folder as the two modules. Import the modules using the following statements:

In [ ]:
import crsutil as crs
import tleplot as tle

To list the names of all the functions and variables defined in the module type `dir(module)` or use `module?` to display the docstring of the module 
```Python
dir(crs)
crs?
```

In [ ]:
dir(crs)

In [ ]:
crs?

To get the docs for a specific function, use `help(functionname)` or `functionname?`. 
```Python
help(crs.orb2vec)
```
Try this out yourself for several functions...

In [ ]:
help(crs.orb2vec)

You see that `crsutil` contains functions for coordinate and time related operations. The ones we are interested in now are the

- GMST and UT1 conversions
- Keplerian elements

The other functions will come in handy as well. 

These are however not the main functions we use. The interesting ones are in `tleplot.py`. To see what is available type
```
dir(tle)
tle?
```
We use `tle` because this is how `tlelot` was imported.

In [ ]:
tle?

The main functions we are going to use are `tleget`, `tleread`, `tle2vec` and `tleplot`. Other functions that are handy are `tlefind` and `tledatenum`.

We explain them in the next sections.

## Download TLE for Earth Resource Satellites from Celestrak

The Two-Line-Elements for a large number of satellites can be downloaded from http://www.celestrak.com/NORAD/elements .

The direct download link for **Earth Resource Satellites** is http://www.celestrak.com/NORAD/elements/resource.txt. 
You can use your favourite browser to download this file and save this file in the current folder. You can also use the function `tleget` to download the file
```
filename = tleget('resource')
```
`filename` will get the value `resource.txt`. If you want to download to another file do something like this
```
filename = tleget('resource','resource-17-Nov-2020.tle')
```
A file called `resource_10-Oct-2017.tle` is already provided in the zip file. You may skip the download step and use this file, but the `resource.txt` downloaded from Celestrak will give you the most up-to-date elements.

In [ ]:
filename = tle.tleget('resource')

See also `help(tle.tleget)`

In [ ]:
help(tle.tleget)

## Read Two-line element files

The function `tleread` reads the NORAD Two-Line-Elements (TLE), which contain the Keplerian elements for Earth orbiting satellites, from file. To find out how it's works type `help(tle.tleread)`.

In [ ]:
help(tle.tleread)

To read the TLE you’ve just downloaded do something like this
```
tleERS =  tle.tleread(filename)
tleERS =  tle.tleread('resource.txt')
```
using the `filename` returned by `tleget` or an explicit filename. The output is a list of named tuples, which we have given the name `tleERS`. You may of course use a different name.

In [ ]:
tleERS=tle.tleread('resource.txt')

During the reading `tleread` prints some useful information about the two-line elements in the file. 

Do you recognize the parameters that are printed? How many minutes is the typical orbital period?

If you don't want all this output you can do use `verbose=0` to suppress the output,
```
tleERS = tle.tleread('resource.txt', verbose=0)
```
The results are saved in a structure array, e.g. `tleERS`, you can choose the name yourself. Check what is inside the structure array by typing `tleERS`, or `tleERS[:2]`, to see the first two entries

In [ ]:
tleERS[:2]

For instance, the mean anomaly is stored in `tle[k].m0` and mean motion in `tle[k].n0`. 
The reference epoch `t0` is stored in two parts, the year in `tle[k].year` and 
day of year in `tle[k].epoch`. This information is not really interesting
as most functions use `tle` and you don't realy care how the information is stored in
the structure array.

In [ ]:
print('satellite, m0:',tleERS[1].name, tleERS[1].m0)
print('satellite, year, epoch:',tleERS[25].name, tleERS[25].year, tleERS[25].epoch)

As you can see there are many Earth Resource satellites stored in `tleERS`. We need to have a method to find particular satellites in the structure array `tleERS`.

## Selecting satellites using `tlefind`

To find particular satellites in `tleERS` use the function `tlefind`. E.g. to find all satellites with ‘RA’ in the name do 
```
isat, satnames = tlefind(tleERS,’RA’)
```
Do `help(tlefind)` to see what the function does. 

To specifically find Radarsat-2 type
```
isat, satnames = tlefind(tleERS,’RADARSAT-2’)
```
If you don't type the full name of the satellite a list with matching satellites is returned.

Try this our yourself, and see what is in the variables that are returned. What is in `isat`?

In [ ]:
isat, satnames = tle.tlefind(tleERS,'RA')
print('isat:',isat)

In [ ]:
isat, satnames = tle.tlefind(tleERS,'RADARSAT-2')
print('isat:',isat)

In [ ]:
help(tle.tlefind)

## Compute satellite position and velocity for Radarsat-2

The function `tle2vec` is used to compute satellite positions and velocity. 
`tle2vec` requires three inputs: 

- the named tuple list `tleERS` with two-line elements, 
- a date, daterange similar to used by `tledatenumber` or an array with Matlab datenumbers `t`, and 
- the satellite identifier. This can be a name or position in `tleERS` 

To obtain the position and velocity at a specific date type e.g.
```
xsat, vsat = tle2vec(tleERS,'2017-10-10 0:00:00','RADARSAT-2')
```
The function is vectorized,  to compute more than one epoch, you can specify multiple epochs. There are basically three ways you can do this

1. Create a vector t using the following code (cumbersome), e.g. 
```
t1 = datenum(‘2017-10-10 0:00:00’);
t = np.arange([t1, t1+1, 1/(24*60)])
```
Make sure that the epoch you choose is within a few days of the reference epoch for the Two-Line-Elements.<br/><br/>

2.	Use  tledatenum to create a vector with times (preferred method), e.g.
```
t = tledatenum(['2013-9-13 0:00:00', 24*60 ,1])
```
Then call  `xsat, vsat = tle2vec(tleERS,t,'RADARSAT-2')`.<br/><br/>

3. Use the `tledatenum` range specification directly in the call to `tle2vec`,
```
xsat, vsat = tle2vec(tleERS, ['2013-9-13 0:00:00', 24*60 ,1],'RADARSAT-2')
```
This form is only useful in case you are not interested in the values of *t* itself.

`tle2vec` returns the satellite position in `xsat` and velocity in `vsat`. `xsat` and `vsat` have shape \[*n*,3\], with *n* the number of epochs in *t*.

Type `help(tle2vec)` to find out more on `tle2vec` and `help(tledatenum)` to find out more on the `tledatenum` range specification. 

In [ ]:
help(tle.tle2vec)

In [ ]:
help(tle.tledatenum)

To compute the position and velocity of RADARSAT-2 you need two things

- the named typle list `tleERS` with two-line elements (including those of RADARSAT-2)
- a daterange specification or array with datenumbers

**Make sure that the datarange specification you choose is within a few days of the reference epoch for the Two-Line-Elements in tleERS for accurate results!**

Below are a couple of examples.

In [ ]:
xsat, vsat = tle.tle2vec(tleERS,['2021-11-15 0:00:00'],'RADARSAT-2')
print('xsat',xsat)
print('vsat',vsat)

In [ ]:
t = tle.tledatenum(['2021-11-15 0:00:00', 24*60 ,1])
xsat, vsat = tle.tle2vec(tleERS,t,'RADARSAT-2')
print('t',t)
print('xsat',xsat)
print('vsat',vsat)

In [ ]:
xsat, vsat = tle.tle2vec(tleERS,['2021-11-15 0:00:00', 24*60 ,1],'RADARSAT-2')
print('xsat',xsat)
print('vsat',vsat)

`tle2vec` returns the satellite position in `xsat` and velocity in `vsat`. `xsat` and `vsat` have shape \[*n*,3\], with *n* the number of epochs in *t*.
`tle2vec` returns arrays with the position and velocity in the ECI reference frame. 

To plot the velocity so something like
```
plt.figure("Velocity")
plt.plot(t, vsat[:, 0]/1000, linewidth=2, label="V_X")
plt.plot(t, vsat[:, 1]/1000, linewidth=2, label="V_Y")
plt.plot(t, vsat[:, 2]/1000, linewidth=2, label="V_Z")
plt.plot(t, velsat/1000, linewidth=2, color='k', label="V")
plt.title("{} velocity in ECI".format(satid))
plt.ylabel("km/s")
plt.xlabel("Date")
plt.legend()
``` 

See for example the code below, where all the necessary packages are imported, including reading of two-line elements.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tleplot as tle

tleERS=tle.tleread('resource.txt',verbose=0)

isat, satnames = tle.tlefind(tleERS,'RADARSAT-2')
t = tle.tledatenum(['2021-11-15 0:00:00', 24*60 ,1])
xsat, vsat = tle.tle2vec(tleERS,t,isat)

print(t.shape)
print(vsat.shape)
plt.figure("Velocity")
plt.plot(t, vsat[:, 0]/1000, linewidth=2, label="V_X")
plt.plot(t, vsat[:, 1]/1000, linewidth=2, label="V_Y")
plt.plot(t, vsat[:, 2]/1000, linewidth=2, label="V_Z")
plt.title("{} velocity in ECI".format(satnames))
plt.ylabel("km/s")
plt.xlabel("Date")
plt.legend() 

Note that in the above we passed to `tle2vec` the position in `tleERS` instead of the name of the satellite. This is sometimes more convenient.

If the satellite name in the call to `tle2vec` is not unique `tle2vec` will complain and print a list of satellites, try
```
  xsat, vsat = tle2vec(tleERS,['2020-11-15 0:00:00'],'SENTINEL-1')
```

In [ ]:
xsat, vsat = tle.tle2vec(tleERS,['2020-11-15 0:00:00'],'SENTINEL-1')

There are two Sentinel-1 satellites, you must select one. Instead of the satellite name you can also give the element number in `tleERS`, as returned by `tlefind`. This is useful if you want to loop over more than one satellite.

In [ ]:
tleERS=tle.tleread('resource.txt', verbose=0)
isats, satnames = tle.tlefind(tleERS, 'SENTINEL-1', verbose=0)
for satname in satnames:
    xsat, vsat = tle.tle2vec(tleERS,['2020-11-15 0:00:00'], str(satname))
    print('satname: {}, pos: {}, vel: {}'.format(satname,xsat,vsat))

for isat in isats:
    xsat, vsat = tle.tle2vec(tleERS,['2020-11-15 0:00:00'], isat)
    print('satnum: {}, pos: {}, vel: {}'.format(isat,xsat,vsat))
    

## Algorithm for computing satellite position and velocity from TLE

Have a look at `tle2vec`and the functions it is calling so see how the computation 
is done. 6.	 Which functions are called and what do they do? 

Make a structure diagram and compare the computations against the lecture notes to put them in context. For instance, which method is the  default propagation model that is used?

The business part of `tle2vec` is
```
# Find the satellite SATID

isat=tlefind(tleERS,'RADARSAT-2') 

# Compute satellite state vectors (position and velocity in ECI)

propagation='J2'                           # Propagation method for orbital elements
orb=tle2orb(tle[isat],t,propagation);      # Matrix of orbit elements
vec=orb2vec(orb);                          # State vector

xsat=vec[:,1:3]
vsat=vec[:,4:6]
```
The function `tle2orb` computes the matrix of orbital (Kepler) elements for the time vector *t* from the two-line elements. The third argument is the *orbit propagation* method. The default orbit propagation method is `J2`, which 
takes the flattening of the Earth into account. The recommended orbit
propagation method for two-line elements is `SGP4`, but this is currently not implemented
in the Python function. The propagation method `NOJ2` ignores the effect 
of J2 on the orbit propagation, which yields very unrealistic results in the long 
term. 

The function `orb2vec` (from `crsutil`) converts the orbital (Kepler) elements into the
state-vector. 

The orbit propagation code for `NOJ2`, ignoring effect of J2, in `tle2orb` is in pseudo code
```
t0 = datenum(tle[isat].year,1,1) + tle[isat].epoch - 1
m = tle[isat].m0 + tle[isat].n0 * (t-t0)          # Mean anomaly 
ignore,nu = keplerm(m, tle[isat].ecc0)            # True anomaly
orb= [ ...                                        # Matrix of orbit elements 
   repmat([ tle[isat].a0 tle[isat].ecc0 tle[isat].inc0 tle[isat].raan0 tle[isat].argp0 ],size(nu)) ...
         nu ];            
```
It uses `keplerm` from the `crsutil` module to solve Kepler's equation and compute from the mean anomaly the true anomaly. The orbital elements are stored in the matrix `orb`; for the first five columns the values in each row are the 
same, only column six contains a time varying variable (the true anomaly). 

## Plotting satellite position, velocity and tracks

The function `tleplot1` can be used to plot the satellite position, velocity and (ground) tracks. 

To view the help on `tleplot1` type
```
help(tleplot1)
```
Actually, `tleplot1` calls `tle2vec` and then plots the results. The first three
arguments are also the same as for `tle2vec1`. The fourth optional argument is new, this is a
vector with the spherical coordinates of the observer which is needed to compute the
the elevation and azimuth of the satellite from an observer.

For example, to plot position, velocity and (ground) tracks you can use the following call
with three input arguments 
```
tleplot1(tleERS,['2020-11-15 0:00:00', 24*60 ,1],'RADARSAT-2')
```
This produces several plots

- plots of satellite positions and velocity in the ECI as function of time,
- plot with the right ascession and declination in ECI, or, satellite track in ECI. Timestamps are plotted for the first epochs, followed by an addtional 15 timestamps for the remainder of the period,
- ground track of the satellite (ECEF, longitude and latitude),
- 3D representations in ECI end ECEF

Below we import the `tleplot` module, read two-line elements for Earth resouce satellites using `tleread`, and then plot using `tleplot1` 

In [ ]:
import tleplot as tle

tleERS=tle.tleread('resource.txt',verbose=0)
tle.tleplot1(tleERS,['2021-11-15 0:00:00', 24*60 ,1],'RADARSAT-2')

Take you time to understand what is plotted. 

Also note that the day you select for plotting is not too far off from the date you downloaded the two-line elements. 

In [ ]:
tle.tleplot1?

## Plotting satellite visibility and elevation angles

To plot the satellite visibility for an observer you have to add a fourth argument with the observer position to `tleplot1`.

For example, to plot position, velcity, ground tracks and satellite visibility for Delft,
use
```
tleplot1(tleERS,['2020-11-15 0:00:00', 24*60 ,1],'RADARSAT-2',[ 52, 4.8,  0 ])
```
This adds two more plots

- plots with the range, range-rate, elevation angle and azimuth angle. When the satellite is not visible the lines are dotted.
- skyplot as seen from the observer

and modifies the other other plots.

In the plot of
the ground tracks the positions that are visible from Delft are plotted
with crosses. Note that in plots that use an ECI referenced frame the 
position of the observer is changing over time.

In [ ]:
tle.tleplot1(tleERS,['2021-11-15 0:00:00', 24*60 ,1],'RADARSAT-2',[ 52, 4.8,  0 ])

You can use these functions to plot the orbits for other satellites with two-line-elements.

You can check out the code in `tleplot1.m` to find out what 
is going on and how the various plots are produced. 


The function `tleplot1` is just a convenience method doing a lot of plots at the same time. For serious work (e.g. your assignment) it is *not* recommended to use `tleplot` directly, but you may reuse some of the code for your purposes. 

[End of this Jupyter notebook]